# Advent of Code Day 2

In [1]:
## Set notebook to auto reload updated modules
from __future__ import annotations

%load_ext autoreload
%autoreload 2

In [2]:
from loguru import logger as log

log.level("ERROR")

Level(name='ERROR', no=40, color='<red><bold>', icon='❌')

In [3]:
import json
from pathlib import Path

import pandas as pd

In [4]:
from aoc_mod.utils import file_utils, nb_utils

In [5]:
def eval_num_pair(num_pair: list[int]) -> bool:
    """Evaluate if num_pair passes all conditions for safety.
    
    Conditions:
    
    - The list must contain at least 1 number.
    - If more than 1 number:
        - All numbers must be either ascending or descending.
        - Adjacent numbers must differ by a minimum of 1 and a maximum of 3.

    Params:
        num_pair (list[int]): A list of numbers to evaluate.

    Returns:
        bool: `True` if all conditions are met, `False` otherwise.
    """
    log.debug(f"Type of num_pair: {type(num_pair)}")
    ## Check if there's at least 1 number in the list
    if len(num_pair) == 0:
        return False

    ## If only one number, return True (no adjacent levels to compare)
    if len(num_pair) == 1:
        return True

    ## Determine if the sequence is ascending or descending
    ascending = all(b > a for a, b in zip(num_pair, num_pair[1:]))
    descending = all(a > b for a, b in zip(num_pair, num_pair[1:]))

    ## Ensure the sequence is either fully ascending or fully descending
    if not (ascending or descending):
        return False

    ## Check that adjacent numbers differ by at least 1 and at most 3
    valid_differences = all(1 <= abs(b - a) <= 3 for a, b in zip(num_pair, num_pair[1:]))

    ## If all conditions are met, return True
    return valid_differences

In [6]:
def re_eval_num_pair(num_pair: list[int]) -> bool:
    """Evaluate if safe_pair passes all conditions for safety on re-evaluation.
    
    Conditions:
        
    - On re-evaluation, removing any 1 number from the list will not alter the safe state.
    
    Params:
      safe_pair (list[int]): A list of integers that was deemed `Safe` on the initial evaluation.
      
    Returns:
        (bool): `True` if safe_pair passes re-evaluation, otherwise `False`.
    
    """
    ## Ensure input list is safe
    initial_status = "Safe" if eval_num_pair(num_pair) else "Unsafe"
    
    if initial_status == "Unsafe":
        return False
    
    ## Iterate through each index in the list
    for i in range(len(num_pair)):
        ## Create a new sub-list by removing the current index
        sub_list = num_pair[:i] + num_pair[i+1:]
        
        ## Re-evaluate the sub-list
        sub_list_status = eval_num_pair(sub_list)

        ## If the status of the sub-list differs from the initial, return False
        if sub_list_status != initial_status:
            return False

    return True


In [7]:
inputs_file = Path("./inputs")
inputs_file.exists()

True

In [8]:
inputs = file_utils.load_inputs("./inputs")
type(inputs)

list

In [9]:
num_pairs: list[list[int]] = []

In [10]:
for line in inputs:
    nums = [int(i) for i in line.strip().split(" ")]
    
    num_pairs.append(nums)
    
display(f"Loaded [{len(num_pairs)}] number pairs from inputs file")

'Loaded [1000] number pairs from inputs file'

## Part 1

In [11]:
evaluated_num_pairs = []

In [12]:
for num_pair in num_pairs:
    # display(f"Pair ({type(num_pair)}[{type(num_pair[0])}]: {num_pair}")
    value = "Safe" if eval_num_pair(num_pair) else "Unsafe"
    evaluated_pair = {"report_safety": value, "numbers": num_pair}
    # display(evaluated_pair)

    evaluated_num_pairs.append(evaluated_pair)

In [13]:
safe_pairs = [p for p in evaluated_num_pairs if p["report_safety"] == "Safe"]
display(f"Found [{len(safe_pairs)}] safe pair(s).")

if len(safe_pairs) >= 0:
    display(f"First 10 safe pairs:")
    for p in safe_pairs[:10]:
        display(p)

'Found [483] safe pair(s).'

'First 10 safe pairs:'

{'report_safety': 'Safe', 'numbers': [33, 34, 35, 36, 39, 42, 45, 48]}

{'report_safety': 'Safe', 'numbers': [69, 70, 72, 73, 75, 78, 80]}

{'report_safety': 'Safe', 'numbers': [53, 50, 49, 48, 47]}

...

{'report_safety': 'Safe', 'numbers': [32, 35, 37, 39, 41]}

{'report_safety': 'Safe', 'numbers': [79, 76, 73, 71, 70, 69, 66]}

{'report_safety': 'Safe', 'numbers': [72, 70, 67, 66, 64, 63, 61]}

{'report_safety': 'Safe', 'numbers': [33, 34, 36, 38, 40, 42, 45, 46]}

{'report_safety': 'Safe', 'numbers': [75, 72, 70, 67, 64]}

In [14]:
part1_solution = {"safe_reports_count": len(safe_pairs) or 0, "safe_reports": safe_pairs, "inputs": {"number_pairs": num_pairs, "evaluated_number_pairs": evaluated_num_pairs}}

## Part 2

In [15]:
evaluated_safe_pairs = []

In [16]:
# Process each pair in the raw list
for num_pair in num_pairs:
    # Re-evaluate the safety status of the pair
    re_evaluation_value = "Safe" if re_eval_num_pair(num_pair) else "Unsafe"
    
    # Create a dictionary for the evaluated pair
    re_evaluated_safe_pair = {"report_safety": re_evaluation_value, "numbers": num_pair}
    
    # Append only unique pairs to the evaluated list
    if re_evaluated_safe_pair not in evaluated_safe_pairs:
        evaluated_safe_pairs.append(re_evaluated_safe_pair)

display(f"Found [{len(evaluated_safe_pairs)}] safe pair(s) on re-evaluation.")

'Found [1000] safe pair(s) on re-evaluation.'